# 4x4 TTT

In [1]:
from Game import *

Version:  0.2.41


In [2]:
def initial_state():
    state=Board(4,4)
    
    return state

In [3]:
def valid_moves(state,player):
    moves = []
    
    if state[0]==0:
        moves.append(0)
    if state[1]==0:
        moves.append(1)
    if state[2]==0:
        moves.append(2)
        
    moves= []
    for location in range(16):
        if state[location]==0:
            moves.append(location)
            
    return moves

In [4]:
state=Board(4,4)
state[2]=1
print(state)
valid_moves(state,1)

 0  0  1  0 
 0  0  0  0 
 0  0  0  0 
 0  0  0  0 



[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [5]:
def update_state(state,player,move):
    new_state=state
    
    new_state[move]=player
    
    return new_state

In [6]:
def four_in_a_row(a,b,c,d,player):
    
    if a==player and b==player and c==player and d==player:
        return True
    else:
        return False
    
    
    
def win_status(state,player):
    
    # 0  1  2  3
    # 4  5  6  7
    # 8  9  10 11
    # 12 13 14 15
    
    if player==1:
        other_player=2
    else:
        other_player=1
    
    if four_in_a_row(state[0],state[1],state[2],state[3],player):
        return 'win'
    if four_in_a_row(state[4],state[5],state[6],state[7],player):
        return 'win'
    if four_in_a_row(state[8],state[9],state[10],state[11],player):
        return 'win'
    if four_in_a_row(state[12],state[13],state[14],state[15],player):
        return 'win'
    if four_in_a_row(state[0],state[5],state[10],state[15],player):
        return 'win'
    if four_in_a_row(state[3],state[6],state[9],state[12],player):
        return 'win'
    if four_in_a_row(state[0],state[4],state[8],state[12],player):
        return 'win'
    if four_in_a_row(state[1],state[5],state[9],state[13],player):
        return 'win'
    if four_in_a_row(state[2],state[6],state[10],state[14],player):
        return 'win'
    if four_in_a_row(state[3],state[7],state[11],state[15],player):
        return 'win'
    if not valid_moves(state,other_player):
        return 'stalemate'
    
    
def show_state(state):
    print(state)
                      

          

In [7]:
def random_move(state,player):    
    moves=valid_moves(state,player)
    return random.choice(moves)
 
random_agent=Agent(random_move) 

In [8]:
def human_move(state,player):    
    print("""
    0  1  2  3
    4  5  6  7
    8  9  10 11
    12 13 14 15
    """)
    move=int(input("What location do you want to move?"))
    return move
 
human_agent=Agent(human_move) 

In [9]:
from Game.minimax import *
def minimax_move(state,player):
    values,moves=minimax_values(state,player,display=True)#set max depth
    
    return top_choice(moves,values)
minimax_agent=Agent(minimax_move)

In [10]:
def skittles_move(state,player,info):
    T=info.T
    last_state=info.last_state
    last_action=info.last_action
    
    
    if state not in T:
        actions=valid_moves(state,player)
        T[state]=Table()
        for action in actions:
            T[state][action]=3  # number of skittles
        
    
    move=weighted_choice(T[state])
    
    if move is None:  # can't win from this state
        if not last_state is None:
            T[last_state][last_action]-=1   # take away a skittle
            if T[last_state][last_action]<0:
                T[last_state][last_action]=0
    
        move=random_move(state,player)
    
    return move

def skittles_after(status,player,info):  # this is called after the game is over
    T=info.T
    last_state=info.last_state
    last_action=info.last_action

    if status=='lose':
        T[last_state][last_action]-=1   # take away a skittle
        if T[last_state][last_action]<0:
            T[last_state][last_action]=0

In [11]:
skittles_agent1=Agent(skittles_move)
skittles_agent1.T=Table()
skittles_agent1.post=skittles_after

skittles_agent2=Agent(skittles_move)
skittles_agent2.T=Table()
skittles_agent2.post=skittles_after

In [12]:
def Q_move(state,player,info):
    Q=info.Q
    last_state=info.last_state
    last_action=info.last_action
    learning=info.learning
    
    α=info.α  # learning rate
    ϵ=info.ϵ  # how often to take a random move
    γ=info.γ  # memory constant -- how quickly does the table update back in time (earlier in the game)
    
    # \alpha <hit tab>    α
    # \epsilon <hit tab>  ϵ
    # \gamma <hit tab>    γ
    
    if state not in Q:
        actions=valid_moves(state,player)
        Q[state]=Table()
        for action in actions:
            Q[state][action]=0  # initial value of table
    
    if learning:
        if random.random()<ϵ:  # take a random move occasionally to explore the environment
            move=random_move(state,player)
        else:
            move=top_choice(Q[state])
    else:
        move=top_choice(Q[state])
    
    if not last_action is None:  # not the first move
        reward=0
        
        # learn
        if learning:
            Q[last_state][last_action]+=α*(reward +
                        γ*max([Q[state][a] for a in Q[state]]) - Q[last_state][last_action])
    
    return move

In [13]:
def Q_after(status,player,info):
    Q=info.Q
    last_state=info.last_state
    last_action=info.last_action
    learning=info.learning
    
    α=info.α  # learning rate
    ϵ=info.ϵ  # how often to take a random move
    γ=info.γ  # memory constant -- how quickly does the table update back in time (earlier in the game)
    
    # \alpha <hit tab>    α
    # \epsilon <hit tab>  ϵ
    # \gamma <hit tab>    γ

    if status=='lose':
        reward=-1
    elif status=='win':
        reward=1
    elif status=='stalemate':
        reward=.5 # value stalemate a little closer to a win
    else:
        reward=0
    
    
    if learning:
        Q[last_state][last_action]+=α*(reward - Q[last_state][last_action])

In [14]:
Q1_agent=Agent(Q_move)
Q1_agent.post=Q_after
Q1_agent.Q=Table()  # makes an empty table
Q1_agent.learning=True

Q1_agent.α=0.3  # learning rate
Q1_agent.ϵ=0.1  # how often to take a random move
Q1_agent.γ=0.9  # memory constant -- how quickly does the table update back in time (earlier in the game)

In [15]:
Q2_agent=Agent(Q_move)
Q2_agent.post=Q_after
Q2_agent.Q=Table()  # makes an empty table
Q2_agent.learning=True

Q2_agent.α=0.3  # learning rate
Q2_agent.ϵ=0.1  # how often to take a random move
Q2_agent.γ=0.9  # memory constant -- how quickly does the table update back in time (earlier in the game)

In [16]:
g=Game()

In [17]:
g.run(random_agent, minimax_agent)

====
Game  1
 0  0  0  0 
 0  0  0  0 
 0  0  0  0 
 0  0  0  0 

Player 1 moves 9
 0  0  0  0 
 0  0  0  0 
 0  1  0  0 
 0  0  0  0 

  Choice Time: 6 minutes 8.969698905944824 seconds 
Player 2 moves 6
 0  0  0  0 
 0  0  2  0 
 0  1  0  0 
 0  0  0  0 

Player 1 moves 8
 0  0  0  0 
 0  0  2  0 
 1  1  0  0 
 0  0  0  0 

  Choice Time: 1 minutes 30.113861083984375 seconds 
Player 2 moves 14
 0  0  0  0 
 0  0  2  0 
 1  1  0  0 
 0  0  2  0 

Player 1 moves 12
 0  0  0  0 
 0  0  2  0 
 1  1  0  0 
 1  0  2  0 

  Choice Time: 13.569037199020386 seconds 
Player 2 moves 5
 0  0  0  0 
 0  2  2  0 
 1  1  0  0 
 1  0  2  0 

Player 1 moves 0
 1  0  0  0 
 0  2  2  0 
 1  1  0  0 
 1  0  2  0 

  Choice Time: 1.141730546951294 seconds 
Player 2 moves 4
 1  0  0  0 
 2  2  2  0 
 1  1  0  0 
 1  0  2  0 

Player 1 moves 1
 1  1  0  0 
 2  2  2  0 
 1  1  0  0 
 1  0  2  0 

  Choice Time: 0.17254996299743652 seconds 
Player 2 moves 2
 1  1  2  0 
 2  2  2  0 
 1  1  0  0 
 1  0  2  0 

[2]

In [18]:
g.run(skittles_agent1, Q2_agent)

====
Game  1
 0  0  0  0 
 0  0  0  0 
 0  0  0  0 
 0  0  0  0 

Player 1 moves 9
 0  0  0  0 
 0  0  0  0 
 0  1  0  0 
 0  0  0  0 

Player 2 moves 11
 0  0  0  0 
 0  0  0  0 
 0  1  0  2 
 0  0  0  0 

Player 1 moves 2
 0  0  1  0 
 0  0  0  0 
 0  1  0  2 
 0  0  0  0 

Player 2 moves 13
 0  0  1  0 
 0  0  0  0 
 0  1  0  2 
 0  2  0  0 

Player 1 moves 10
 0  0  1  0 
 0  0  0  0 
 0  1  1  2 
 0  2  0  0 

Player 2 moves 1
 0  2  1  0 
 0  0  0  0 
 0  1  1  2 
 0  2  0  0 

Player 1 moves 12
 0  2  1  0 
 0  0  0  0 
 0  1  1  2 
 1  2  0  0 

Player 2 moves 6
 0  2  1  0 
 0  0  2  0 
 0  1  1  2 
 1  2  0  0 

Player 1 moves 15
 0  2  1  0 
 0  0  2  0 
 0  1  1  2 
 1  2  0  1 

Player 2 moves 7
 0  2  1  0 
 0  0  2  2 
 0  1  1  2 
 1  2  0  1 

Player 1 moves 5
 0  2  1  0 
 0  1  2  2 
 0  1  1  2 
 1  2  0  1 

Player 2 moves 0
 2  2  1  0 
 0  1  2  2 
 0  1  1  2 
 1  2  0  1 

Player 1 moves 3
 2  2  1  1 
 0  1  2  2 
 0  1  1  2 
 1  2  0  1 

Player 2 moves 4
 2

[0]